# Task 2: Data Cleaning and Transformation

This notebook documents the second phase of the **EthioMed Data Warehouse Project**, focusing on data cleaning and transformation. The aim of this task is to ensure the quality and consistency of the scraped data for storage and further analysis. The following steps are performed:

1. **Data Cleaning**: 
   - Removal of duplicate records.
   - Handling of missing values through appropriate imputation or exclusion.
   - Standardization of formats (e.g., date formats, text normalization).
   - Validation of data integrity against defined standards.

2. **Storing Cleaned Data**: 
   - The cleaned data is stored in a structured database for future queries and reporting.

3. **DBT for Data Transformation**:
   - Setting up DBT (Data Build Tool) for transformation tasks.
   - Defining and running models to transform the cleaned data.
   - Testing and documenting transformations for data quality assurance.

4. **Monitoring and Logging**:
   - Implementing logging to monitor the scraping, cleaning, and transformation process, ensuring traceability of issues.

This task is carried out independently, with the ultimate goal of ensuring a reliable data pipeline for the EthioMed Data Warehouse.


# Importing Liraries

In [1]:
import sys
import os

# Add the scripts folder to the Python path
sys.path.append(os.path.abspath('../scripts'))

In [2]:
from data_cleaning_transformation import *

In [3]:
import data_cleaning_transformation

In [4]:
# Loading the Scraped Dataframe
file_path = "../data/telegram_data.csv"
df = data_cleaning_transformation.load_data(file_path)

In [5]:
df.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,Doctors Ethiopia,@DoctorsET,864,https://youtu.be/5DBoEm-8kmA?si=LDLuEecNfULJVD...,2023-12-18 17:04:02+00:00,NaN
1,Doctors Ethiopia,@DoctorsET,863,ዶክተርስ ኢትዮጵያ በ አዲስ አቀራረብ በ ቴሌቪዥን ፕሮግራሙን ለመጀመር ከ...,2023-11-03 16:14:39+00:00,NaN
2,Doctors Ethiopia,@DoctorsET,862,ሞት በስኳር \n\nለልጆቻችን የምናሲዘው ምሳቃ ሳናቀው እድሚያቸውን ይቀን...,2023-10-02 16:37:39+00:00,NaN
3,Doctors Ethiopia,@DoctorsET,861,ከ HIV የተፈወሰ ሰው አጋጥሟችሁ ያቃል ? ፈውስ እና ህክምና ?\n\nሙ...,2023-09-16 07:54:32+00:00,NaN
4,Doctors Ethiopia,@DoctorsET,860,በቅርብ ጊዜ በሃገራችን ላይ እየተስተዋለ ያለ የተመሳሳይ ፆታ ( Homos...,2023-09-01 16:16:15+00:00,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6288 entries, 0 to 6287
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Channel Title     6288 non-null   object
 1   Channel Username  6288 non-null   object
 2   ID                6288 non-null   int64 
 3   Message           5118 non-null   object
 4   Date              6288 non-null   object
 5   Media Path        4749 non-null   object
dtypes: int64(1), object(5)
memory usage: 294.9+ KB


# Data Cleaning

Removing Duplicates

In [7]:
# Remove duplicates
df = df.drop_duplicates()

Handling Missing Values

In [8]:
# Checking for missing Value
missing_values_table(df)

Your selected dataframe has 6 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Media Path,1539,24.5,object
Message,1170,18.6,object


In [9]:
# Fill missing Media Path values with a placeholder
df['Media Path'] = df['Media Path'].fillna('No media')  # Reassign the column

# Drop rows where Message is NaN
df = df.dropna(subset=['Message'])  # Reassign the DataFrame

In [10]:
# Checking for missing Value again to ensure 
missing_values_table(df)

Your selected dataframe has 6 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,Dtype


Standardizing Formats

In [11]:
# Standardize date formats
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

Removing Emojis

In [12]:

df['Message'] = df['Message'].apply(remove_emojis)

In [15]:
df.columns

Index(['Channel Title', 'Channel Username', 'ID', 'Message', 'Date',
       'Media Path'],
      dtype='object')

In [13]:
summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Channel Title,object,0,5118,4,"{'Lobelia pharmacy and cosmetics': 2254, 'ETHI..."
1,Channel Username,object,0,5118,4,"{'@lobelia4cosmetics': 2254, '@EAHCI': 1528, '..."
2,ID,int64,0,5118,4182,"{71: 4, 864: 3, 463: 3, 467: 3, 469: 3, 470: 3..."
3,Message,object,0,5118,2819,{'NIDO USA 2.2KG Price 5900 birr Telegram htt...
4,Date,"datetime64[ns, UTC]",0,5118,3314,"{2024-10-08 04:35:59+00:00: 24, 2024-10-02 08:..."
5,Media Path,object,0,5118,4031,"{'No media': 1088, 'photos\@yetenaweg_627.jpg'..."


Storing Cleaned Data

In [14]:
store_cleaned_data(df, "ethio_med_data")